# Load Libraries and Set Settings

In [1]:
# Libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import PyPDF2
from io import BytesIO
from tqdm import tqdm
import time
import csv
import sys
from random import randint

# Other Settings
pd.set_option('display.max.colwidth', None) # max display width

# Load Data

In [2]:
# Load data from CSV file into a DataFrame
pdf_content = pd.read_csv('pdf_data.csv')

In [3]:
# Load data from CSV file into a DataFrame
article_link_directory = pd.read_csv('article_link_directory.csv')

In [4]:
len(pdf_content)

27079

In [5]:
pdf_content.head(1)

PDF URL  \
0  https://www.regeringen.se/contentassets/c9981cbd1c054e3fa1a2a917d3e22939/inbjudan-till-larosatena-att-inkomma-med-underlag-till-regeringens-forskningspolitik.pdf   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [6]:
pdf_content.tail(1)

,PDF URL,Inside PDF Text
27078,NO CONTENT,NO CONTENT


In [7]:
len(article_link_directory)

27079

In [8]:
article_link_directory.head(1)

,Content Links,Publishing Info,Collected Links,Outside PDF Text,Full Collected Links
0,https://www.regeringen.se/remisser/2023/11/inbjudan-till-larosaten-myndigheter-och-organisationer-att-inkomma-med-synpunkter-till-regeringens-forskningspolitik/,Publicerad02 november 2023· Uppdaterad10 november 2023·RemissfrånUtbildningsdepartementet,/contentassets/c9981cbd1c054e3fa1a2a917d3e22939/inbjudan-till-larosatena-att-inkomma-med-underlag-till-regeringens-forskningspolitik.pdf,NO CONTENT,https://www.regeringen.se/contentassets/c9981cbd1c054e3fa1a2a917d3e22939/inbjudan-till-larosatena-att-inkomma-med-underlag-till-regeringens-forskningspolitik.pdf


In [9]:
article_link_directory.tail(1)

,Content Links,Publishing Info,Collected Links,Outside PDF Text,Full Collected Links
27078,https://www.regeringen.se/internationella-mr-granskningar-av-sverige/2007/04/cat-kommittens-granskning-2007-04-30-i-malet-e.r.k.-och-y.k.-mot-sverige,"Publicerad30 april 2007·Internationella MR-granskningar av SverigefrånJustitiedepartementet,Utrikesdepartementet",NO CONTENT,"Publicerad30 april 2007\nFN:s CAT-kommitté har i ett yttrande prövat Sveriges efterlevnad av FN:s konvention mot tortyr (CAT). Regeringen publicerar en länk till yttrandet i fulltext samt en sammanfattning av yttrandet.\nMålet rörde ursprungligen avvisning av tre bröder till Azerbajdzjan. Sedan den ena brodern och hans familj beviljats permanent uppehållstillstånd i Sverige avskrev kommittén klagomålet avseende den brodern. De två kvarvarande klagandena gjorde inför kommittén gällande att de i hemlandet riskerade att utsättas för tortyr eller annan behandling i strid med artikel 3 i konventionen till följd av en av brödernas påstådda politiska aktiviteter och medlemskap i det politiska oppositionspartiet ADP. Till stöd för klagomålet ingavs ett stort antal handlingar, bl.a. kallelser till polisförhör. Enligt kommittén visade den ambassadutredning som gjorts i målet på ett övertygande sätt att handlingarna var falska. Kommittén konstaterade att klagandena inte hade kunnat lägga fram motbevis och ansåg att de inte heller i övrigt gjort sannolikt att de skulle utsättas för behandling i strid med konventionen vid ett återvändande till hemlandet. Kommittén fann därför att en avvisning av bröderna till Azerbajdzjan inte skulle strida mot konventionen.\nKonventionen antogs år 1984. FN:s CAT-kommitté övervakar att konventionen respekteras. Kommittén består av oberoende experter. Granskningen sker genom prövning av enskilda klagomål och av återkommande rapportering från konventionsstaterna.\nPublicerad17 november 2023·TalfrånUlf Kristersson,Statsrådsberedningen\nPublicerad17 november 2023·UttalandefrånRomina Pourmokhtari,Klimat- och näringslivsdepartementet\nPublicerad17 november 2023·PressmeddelandefrånNiklas Wykman,Finansdepartementet\nPublicerad17 november 2023·RemissfrånFinansdepartementet\nPublicerad17 november 2023·Departementsserien och promemorior,Rättsliga dokumentfrånFinansdepartementet\nRegeringskansliet103 33 StockholmVäxel 08-405 10 00\nDet här är regeringens och Regeringskansliets gemensamma webbplats.",NO CONTENT


# Combine data

### Combine Data Frames

In [10]:
# Add Inside PDF Text to article_link_directory
article_link_directory['Inside PDF Text'] = pdf_content['Inside PDF Text']

In [11]:
len(article_link_directory)

27079

In [12]:
# Confirm no repeats
article_link_directory['Content Links'].nunique()

27077

In [13]:
article_link_directory.head(1)

Content Links  \
0  https://www.regeringen.se/remisser/2023/11/inbjudan-till-larosaten-myndigheter-och-organisationer-att-inkomma-med-synpunkter-till-regeringens-forskningspolitik/   

                                                                             Publishing Info  \
0  Publicerad02 november 2023· Uppdaterad10 november 2023·RemissfrånUtbildningsdepartementet   

                                                                                                                            Collected Links  \
0  /contentassets/c9981cbd1c054e3fa1a2a917d3e22939/inbjudan-till-larosatena-att-inkomma-med-underlag-till-regeringens-forskningspolitik.pdf   

  Outside PDF Text  \
0       NO CONTENT   

                                                                                                                                                Full Collected Links  \
0  https://www.regeringen.se/contentassets/c9981cbd1c054e3fa1a2a917d3e22939/inbjudan-till-larosatena-att-inkomma-med-underlag-till-regeringens-forskningspolitik.pdf   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [14]:
article_link_directory.tail(1)

,Content Links,Publishing Info,Collected Links,Outside PDF Text,Full Collected Links,Inside PDF Text
27078,https://www.regeringen.se/internationella-mr-granskningar-av-sverige/2007/04/cat-kommittens-granskning-2007-04-30-i-malet-e.r.k.-och-y.k.-mot-sverige,"Publicerad30 april 2007·Internationella MR-granskningar av SverigefrånJustitiedepartementet,Utrikesdepartementet",NO CONTENT,"Publicerad30 april 2007\nFN:s CAT-kommitté har i ett yttrande prövat Sveriges efterlevnad av FN:s konvention mot tortyr (CAT). Regeringen publicerar en länk till yttrandet i fulltext samt en sammanfattning av yttrandet.\nMålet rörde ursprungligen avvisning av tre bröder till Azerbajdzjan. Sedan den ena brodern och hans familj beviljats permanent uppehållstillstånd i Sverige avskrev kommittén klagomålet avseende den brodern. De två kvarvarande klagandena gjorde inför kommittén gällande att de i hemlandet riskerade att utsättas för tortyr eller annan behandling i strid med artikel 3 i konventionen till följd av en av brödernas påstådda politiska aktiviteter och medlemskap i det politiska oppositionspartiet ADP. Till stöd för klagomålet ingavs ett stort antal handlingar, bl.a. kallelser till polisförhör. Enligt kommittén visade den ambassadutredning som gjorts i målet på ett övertygande sätt att handlingarna var falska. Kommittén konstaterade att klagandena inte hade kunnat lägga fram motbevis och ansåg att de inte heller i övrigt gjort sannolikt att de skulle utsättas för behandling i strid med konventionen vid ett återvändande till hemlandet. Kommittén fann därför att en avvisning av bröderna till Azerbajdzjan inte skulle strida mot konventionen.\nKonventionen antogs år 1984. FN:s CAT-kommitté övervakar att konventionen respekteras. Kommittén består av oberoende experter. Granskningen sker genom prövning av enskilda klagomål och av återkommande rapportering från konventionsstaterna.\nPublicerad17 november 2023·TalfrånUlf Kristersson,Statsrådsberedningen\nPublicerad17 november 2023·UttalandefrånRomina Pourmokhtari,Klimat- och näringslivsdepartementet\nPublicerad17 november 2023·PressmeddelandefrånNiklas Wykman,Finansdepartementet\nPublicerad17 november 2023·RemissfrånFinansdepartementet\nPublicerad17 november 2023·Departementsserien och promemorior,Rättsliga dokumentfrånFinansdepartementet\nRegeringskansliet103 33 StockholmVäxel 08-405 10 00\nDet här är regeringens och Regeringskansliets gemensamma webbplats.",NO CONTENT,NO CONTENT


### Create a new column "Text" holding either inside or outside pdf text for each document

In [15]:
# Create a new column "Text" in article_link_directory based on conditions
article_link_directory['Text'] = article_link_directory.apply(lambda row: row['Inside PDF Text'] if row['Outside PDF Text'] == 'NO CONTENT' else row['Outside PDF Text'], axis=1)

In [16]:
article_link_directory.head(1)

Content Links  \
0  https://www.regeringen.se/remisser/2023/11/inbjudan-till-larosaten-myndigheter-och-organisationer-att-inkomma-med-synpunkter-till-regeringens-forskningspolitik/   

                                                                             Publishing Info  \
0  Publicerad02 november 2023· Uppdaterad10 november 2023·RemissfrånUtbildningsdepartementet   

                                                                                                                            Collected Links  \
0  /contentassets/c9981cbd1c054e3fa1a2a917d3e22939/inbjudan-till-larosatena-att-inkomma-med-underlag-till-regeringens-forskningspolitik.pdf   

  Outside PDF Text  \
0       NO CONTENT   

                                                                                                                                                Full Collected Links  \
0  https://www.regeringen.se/contentassets/c9981cbd1c054e3fa1a2a917d3e22939/inbjudan-till-larosatena-att-inkomma-med-underlag-till-regeringens-forskningspolitik.pdf   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [17]:
article_link_directory.tail(1)

,Content Links,Publishing Info,Collected Links,Outside PDF Text,Full Collected Links,Inside PDF Text,Text
27078,https://www.regeringen.se/internationella-mr-granskningar-av-sverige/2007/04/cat-kommittens-granskning-2007-04-30-i-malet-e.r.k.-och-y.k.-mot-sverige,"Publicerad30 april 2007·Internationella MR-granskningar av SverigefrånJustitiedepartementet,Utrikesdepartementet",NO CONTENT,"Publicerad30 april 2007\nFN:s CAT-kommitté har i ett yttrande prövat Sveriges efterlevnad av FN:s konvention mot tortyr (CAT). Regeringen publicerar en länk till yttrandet i fulltext samt en sammanfattning av yttrandet.\nMålet rörde ursprungligen avvisning av tre bröder till Azerbajdzjan. Sedan den ena brodern och hans familj beviljats permanent uppehållstillstånd i Sverige avskrev kommittén klagomålet avseende den brodern. De två kvarvarande klagandena gjorde inför kommittén gällande att de i hemlandet riskerade att utsättas för tortyr eller annan behandling i strid med artikel 3 i konventionen till följd av en av brödernas påstådda politiska aktiviteter och medlemskap i det politiska oppositionspartiet ADP. Till stöd för klagomålet ingavs ett stort antal handlingar, bl.a. kallelser till polisförhör. Enligt kommittén visade den ambassadutredning som gjorts i målet på ett övertygande sätt att handlingarna var falska. Kommittén konstaterade att klagandena inte hade kunnat lägga fram motbevis och ansåg att de inte heller i övrigt gjort sannolikt att de skulle utsättas för behandling i strid med konventionen vid ett återvändande till hemlandet. Kommittén fann därför att en avvisning av bröderna till Azerbajdzjan inte skulle strida mot konventionen.\nKonventionen antogs år 1984. FN:s CAT-kommitté övervakar att konventionen respekteras. Kommittén består av oberoende experter. Granskningen sker genom prövning av enskilda klagomål och av återkommande rapportering från konventionsstaterna.\nPublicerad17 november 2023·TalfrånUlf Kristersson,Statsrådsberedningen\nPublicerad17 november 2023·UttalandefrånRomina Pourmokhtari,Klimat- och näringslivsdepartementet\nPublicerad17 november 2023·PressmeddelandefrånNiklas Wykman,Finansdepartementet\nPublicerad17 november 2023·RemissfrånFinansdepartementet\nPublicerad17 november 2023·Departementsserien och promemorior,Rättsliga dokumentfrånFinansdepartementet\nRegeringskansliet103 33 StockholmVäxel 08-405 10 00\nDet här är regeringens och Regeringskansliets gemensamma webbplats.",NO CONTENT,NO CONTENT,"Publicerad30 april 2007\nFN:s CAT-kommitté har i ett yttrande prövat Sveriges efterlevnad av FN:s konvention mot tortyr (CAT). Regeringen publicerar en länk till yttrandet i fulltext samt en sammanfattning av yttrandet.\nMålet rörde ursprungligen avvisning av tre bröder till Azerbajdzjan. Sedan den ena brodern och hans familj beviljats permanent uppehållstillstånd i Sverige avskrev kommittén klagomålet avseende den brodern. De två kvarvarande klagandena gjorde inför kommittén gällande att de i hemlandet riskerade att utsättas för tortyr eller annan behandling i strid med artikel 3 i konventionen till följd av en av brödernas påstådda politiska aktiviteter och medlemskap i det politiska oppositionspartiet ADP. Till stöd för klagomålet ingavs ett stort antal handlingar, bl.a. kallelser till polisförhör. Enligt kommittén visade den ambassadutredning som gjorts i målet på ett övertygande sätt att handlingarna var falska. Kommittén konstaterade att klagandena inte hade kunnat lägga fram motbevis och ansåg att de inte heller i övrigt gjort sannolikt att de skulle utsättas för behandling i strid med konventionen vid ett återvändande till hemlandet. Kommittén fann därför att en avvisning av bröderna till Azerbajdzjan inte skulle strida mot konventionen.\nKonventionen antogs år 1984. FN:s CAT-kommitté övervakar att konventionen respekteras. Kommittén består av oberoende experter. Granskningen sker genom prövning av enskilda klagomål och av återkommande rapportering från konventionsstaterna.\nPublicerad17 november 2023·TalfrånUl

# Clean up the data to prepare for text analysis

### Replace newline characters in text columns

In [18]:
# Clean the text data in 'Inside PDF Text' and 'Text' and 'Outside PDF Text' columns by replacing newline characters
article_link_directory['Inside PDF Text'] = article_link_directory['Inside PDF Text'].str.replace('\n', ' ')
article_link_directory['Text'] = article_link_directory['Text'].str.replace('\n', ' ')
article_link_directory['Outside PDF Text'] = article_link_directory['Outside PDF Text'].str.replace('\n', ' ')

### Extract date text

In [19]:
# Extract document date from article_link_directory['Publishing Info']
# List to store extracted dates
Date = []

# Iterate through the 'Publishing Dates' column and extract dates based on the condition
# "Updated" dates are disregarded because they only seem to occur with certain documents of lesser importance and don't override the pdf included in content...so the pdf data reflects the original date

for date_string in article_link_directory['Publishing Info']:
    if 'Publicerad' in date_string:
        start_index = date_string.find('Publicerad') + len('Publicerad')
        end_index = date_string.find('·', start_index)
        if end_index != -1:
            extracted_date = date_string[start_index:end_index].strip()
            Date.append(extracted_date)
        else:
            # Handle the case when there's no '·' after 'Publicerad'
            Date.append(None)
    else:
        # Handle the case when 'Publicerad' is not mentioned in the string
        Date.append(None)

In [20]:
# Add the 'Date' list as a new column in the DataFrame
article_link_directory['Date'] = Date

### Extract document type text

In [21]:
# Extract document types from article_link_directory['Publishing Info']
# List to store extracted document types
Document_Type = []

# Iterate through the 'Publishing Info' column and extract document types based on the condition
for date_string in article_link_directory['Publishing Info']:
    last_dot_index = date_string.rfind('·')  # Find the last instance of "·"
    if last_dot_index != -1:
        # Extract string after the last "·"
        extracted_text = date_string[last_dot_index + 1:].strip()
        if 'från' in extracted_text:
            # Extract string before "från"
            extracted_text = extracted_text[:extracted_text.find('från')].strip()
            Document_Type.append(extracted_text)
        else:
            Document_Type.append(None)  # If "från" is not found, append None
    else:
        # Handle the case when there's no "·" in the string
        Document_Type.append(None)

In [22]:
# Add the 'Document_Type' list as a new column in the DataFrame
article_link_directory['Document Type'] = Document_Type

### Extract document source text

In [23]:
# Extract document source from article_link_directory['Publishing Info']
# List to store extracted sources
Source = []

# Iterate through the 'Publishing Info' column and extract sources based on the condition
for date_string in article_link_directory['Publishing Info']:
    # Find the index of "från"
    from_index = date_string.find('från')
    if from_index != -1:
        # Extract string after "från" and add it to the Source list
        extracted_source = date_string[from_index + len('från'):].strip()
        Source.append(extracted_source)
    else:
        # If "från" is not found, append None to indicate no source
        Source.append(None)

In [24]:
# Add the 'Source' list as a new row in the DataFrame
article_link_directory['Source'] = Source

### Remove unnecessary columns

In [25]:
# Identify columns to drop
columns_to_drop = ['Publishing Info', 'Collected Links', 'Outside PDF Text', 'Inside PDF Text']

# Drop the columns
article_link_directory = article_link_directory.drop(columns=columns_to_drop)

### Create a PDF Indicator variable

In [26]:
# Replace 'Full Collected Links' with the actual column name in your DataFrame
article_link_directory['PDF Indicator'] = (article_link_directory['Full Collected Links'] != 'NO CONTENT').astype(int)

### Rename link-containing variables so that they're more descriptive and precise

In [27]:
# Replace the column names with the actual names in your DataFrame
article_link_directory.rename(columns={'Full Collected Links': 'PDF Link', 'Content Links': 'Content Page'}, inplace=True)

### Re-order columns

In [28]:
# Specify new column order
column_order = ['Date', 'Document Type', 'Source', 'PDF Indicator', 'Text', 'Content Page', 'PDF Link']

# Set new column order
article_link_directory = article_link_directory[column_order]

In [29]:
article_link_directory.tail(1)

,Date,Document Type,Source,PDF Indicator,Text,Content Page,PDF Link
27078,30 april 2007,Internationella MR-granskningar av Sverige,"Justitiedepartementet,Utrikesdepartementet",0,"Publicerad30 april 2007 FN:s CAT-kommitté har i ett yttrande prövat Sveriges efterlevnad av FN:s konvention mot tortyr (CAT). Regeringen publicerar en länk till yttrandet i fulltext samt en sammanfattning av yttrandet. Målet rörde ursprungligen avvisning av tre bröder till Azerbajdzjan. Sedan den ena brodern och hans familj beviljats permanent uppehållstillstånd i Sverige avskrev kommittén klagomålet avseende den brodern. De två kvarvarande klagandena gjorde inför kommittén gällande att de i hemlandet riskerade att utsättas för tortyr eller annan behandling i strid med artikel 3 i konventionen till följd av en av brödernas påstådda politiska aktiviteter och medlemskap i det politiska oppositionspartiet ADP. Till stöd för klagomålet ingavs ett stort antal handlingar, bl.a. kallelser till polisförhör. Enligt kommittén visade den ambassadutredning som gjorts i målet på ett övertygande sätt att handlingarna var falska. Kommittén konstaterade att klagandena inte hade kunnat lägga fram motbevis och ansåg att de inte heller i övrigt gjort sannolikt att de skulle utsättas för behandling i strid med konventionen vid ett återvändande till hemlandet. Kommittén fann därför att en avvisning av bröderna till Azerbajdzjan inte skulle strida mot konventionen. Konventionen antogs år 1984. FN:s CAT-kommitté övervakar att konventionen respekteras. Kommittén består av oberoende experter. Granskningen sker genom prövning av enskilda klagomål och av återkommande rapportering från konventionsstaterna. Publicerad17 november 2023·TalfrånUlf Kristersson,Statsrådsberedningen Publicerad17 november 2023·UttalandefrånRomina Pourmokhtari,Klimat- och näringslivsdepartementet Publicerad17 november 2023·PressmeddelandefrånNiklas Wykman,Finansdepartementet Publicerad17 november 2023·RemissfrånFinansdepartementet Publicerad17 november 2023·Departementsserien och promemorior,Rättsliga dokumentfrånFinansdepartementet Regeringskansliet103 33 StockholmVäxel 08-405 10 00 Det här är regeringens och Regeringskansliets gemensamma webbplats.",https://www.regeringen.se/internationella-mr-granskningar-av-sverige/2007/04/cat-kommittens-granskning-2007-04-30-i-malet-e.r.k.-och-y.k.-mot-sverige,NO CONTENT


# Export cleaned and organized data

In [30]:
# Make a test data set
new_dataframe = article_link_directory.head(25)

In [34]:
len(new_dataframe)

25

In [32]:
new_dataframe.head(1)

Date Document Type                    Source  PDF Indicator  \
0  02 november 2023        Remiss  Utbildningsdepartementet              1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [35]:
# Download .csv of the data
new_dataframe.to_csv('unclassified_data.csv', index=True)

# Test data from the Sentiment Analysis!

In [36]:
# Load data from CSV file into a DataFrame
classified_data = pd.read_csv('classified_data.csv')

In [39]:
len(classified_data)

25

In [40]:
classified_data

,Unnamed: 0,ID,Date,Fear 0 Sen,Fear 1 Sen,Fear 2 Sen,Violence 0 Sen,Violence 1 Sen,Violence 2 Sen,Document Type,...,Integrationssvårigheter,Diskriminering,Integrationstjänster,Month,Year,Keyword Present,Keywords,Sentence FVS,Article FVS,Average FVS
0,0,0,2023-11-02,0.917277,0.076140,0.006583,0.996119,0.001370,0.002511,Remiss,...,0,0,0,11,2023,0,[],4.330190,1.331967,2.831078
1,1,1,2023-11-10,0.972475,0.024549,0.002975,0.999160,0.000354,0.000486,Regeringsuppdrag,...,0,0,0,11,2023,0,[],1.418251,3.044924,2.231587
2,2,2,2023-06-27,0.996270,0.002704,0.001027,0.998764,0.000592,0.000644,Remiss,...,0,1,0,6,2023,1,"['Integration', 'Segregation', 'Migrationsverket', 'Diskriminering']",0.248309,0.884333,0.566321
3,3,3,2023-09-08,0.995365,0.003028,0.001607,0.998342,0.000822,0.000836,Remiss,...,0,0,0,9,2023,0,[],0.314632,0.620949,0.467791
4,4,4,2023-07-07,0.996650,0.002474,0.000877,0.998282,0.000872,0.000846,Remiss,...,0,0,0,7,2023,1,['Migrationsverket'],0.253385,0.930512,0.591948
5,5,5,2023-11-09,0.984609,0.012445,0.002946,0.993505,0.002831,0.003664,"Kommittédirektiv,Rättsliga dokument",...,0,0,0,11,2023,0,[],1.094288,0.293672,0.693980
6,6,6,2023-11-09,0.996961,0.002150,0.000890,0.998830,0.000616,0.000555,Remiss,...,0,0,0,11,2023,0,[],0.210477,0.400779,0.305628
7,7,7,2023-11-09,0.985052,0.013266,0.001682,0.997709,0.000825,0.001465,"Departementsserien och promemorior,Rättsliga dokument",...,0,0,0,11,2023,0,[],0.861943,0.740281,0.801112
8,8,8,2023-11-09,0.751052,0.162137,0.086811,0.993547,0.002729,0.003724,"Lagrådsremiss,Rättsliga dokument",...,0,0,0,11,2023,1,['Anpassning'],12.770064,0.924182,6.847123
9,9,9,2023-11-09,0.965779,0.031272,0.002950,0.995897,0.001580,0.002523,"Lagrådsremiss,Rättsliga dokument",...,0,0,0,11,2023,0,[],1.916229,0.984535,1.450382
